## Transforming the data in the final stages

In [0]:
from pyspark.sql.functions import (
    col, to_timestamp, to_date, when, lit, sum as _sum, first, coalesce, date_sub, current_timestamp
)

df_news_silver = spark.table("crypto_sentiment_silver_news")

df_sentiment_daily = df_news_silver.groupBy("symbol", "name", "news_date").agg(
    _sum("sentiment_score").alias("total_daily_score"),
    first("news_headline").alias("top_headline") # Top headline for context
)


display(df_sentiment_daily)

symbol,name,news_date,total_daily_score,top_headline
BTC,BITCOIN,2026-02-07,-2,"Discussion on the end of the monetary system and collapsing fiat currencies, implying a positive outlook for Bitcoin as an alternative."
DOGE,DOGECOIN,2026-02-07,1,Major indicators like Bollinger Bands hint the downtrend is almost over for Dogecoin.
ENS,ETHEREUM NAME SERVICE,2026-02-07,-1,ENS drops its L2.
ETH,ETHEREUM,2026-02-07,1,"Vitalik Buterin calls for L2 shift as Ethereum L1 scales, indicating continued development and scaling efforts."
ONDO,ONDO,2026-02-07,1,"21Shares advanced an Ondo ETF filing, and the token price shows signs of recovery."
SHIB,SHIBA INU,2026-02-07,-1,The Shiba Inu team issued a crucial wallet security notice to the SHIB community due to a new emerging threat.
SOL,SOLANA,2026-02-07,1,"Solana (SOL) staged a sharp intraday recovery, posting a 12% daily gain despite lingering market uncertainty, though long-term holder buying momentum is slowing."


In [0]:
    df_market_silver = spark.table("crypto_sentiment_silver_price")

    df_gold = df_market_silver.join(
        df_sentiment_daily,
        (df_market_silver.symbol == df_sentiment_daily.symbol) & 
        (date_sub(df_market_silver.market_date, 1) == df_sentiment_daily.news_date),
        how="left"
    ).drop(df_sentiment_daily.symbol)

    df_gold = df_gold.withColumn("total_daily_score", coalesce(col("total_daily_score"), lit(0)))

In [0]:
df_gold = df_gold.withColumn(
    "trade_signal",
    # SCENARIO 1: Strong Buy (Price Up + Positive News + High Vol)
    when(
        (col("percent_change_24h") > 0) & 
        (col("total_daily_score") > 0) & 
        (col("volume_24h") > 1000000), 
        "STRONG BUY"
    )
    # SCENARIO 2: Buy the Dip (Price Down + Positive News)
    .when(
        (col("percent_change_24h") < -2) & 
        (col("total_daily_score") > 0), 
        "VALUE BUY (Dip)"
    )
    # SCENARIO 3: Bearish Divergence (Price Up + Negative News) -> Watch out!
    .when(
        (col("percent_change_24h") > 0) & 
        (col("total_daily_score") < 0), 
        "WARNING (Fakeout)"
    )
    # SCENARIO 4: Panic Sell (Price Down + Negative News)
    .when(
        (col("percent_change_24h") < 0) & 
        (col("total_daily_score") < 0), 
        "STRONG SELL"
    )
    .otherwise("HOLD / NEUTRAL")
)

display(df_gold)

symbol,price,cmc_rank,volume_24h,percent_change_1h,percent_change_24h,percent_change_7d,market_timestamp,ingested_at,market_date,risk_category,name,news_date,total_daily_score,top_headline,trade_signal
BTC,71184.865,1,44447605920,-0.31,3.23,-8.55,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Blue Chip,BITCOIN,2026-02-07,-2,"Discussion on the end of the monetary system and collapsing fiat currencies, implying a positive outlook for Bitcoin as an alternative.",WARNING (Fakeout)
ONDO,0.2618,51,72483716,0.47,2.51,-6.67,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Small Cap,ONDO,2026-02-07,1,"21Shares advanced an Ondo ETF filing, and the token price shows signs of recovery.",STRONG BUY
ETH,2118.6295,2,32943822587,-0.63,3.75,-10.33,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Blue Chip,ETHEREUM,2026-02-07,1,"Vitalik Buterin calls for L2 shift as Ethereum L1 scales, indicating continued development and scaling efforts.",STRONG BUY
LINK,8.9258,16,720055017,-0.72,1.7,-7.94,2026-02-08T14:31:00Z,2026-02-08T14:31:43Z,2026-02-08,Mid Cap,null,null,0,null,HOLD / NEUTRAL
DOT,1.3629,33,127304972,-0.08,1.39,-10.13,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Mid Cap,null,null,0,null,HOLD / NEUTRAL
USDT,0.9993,3,94096940583,-0.01,0.04,0.03,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Blue Chip,null,null,0,null,HOLD / NEUTRAL
AVAX,9.2433,24,304212062,-0.15,1.3,-7.32,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Mid Cap,null,null,0,null,HOLD / NEUTRAL
XRP,1.4598,4,3228554522,-0.17,3.9,-9.55,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Blue Chip,null,null,0,null,HOLD / NEUTRAL
ADA,0.2738,11,1061909859,-0.38,1.04,-6.09,2026-02-08T14:31:00Z,2026-02-08T14:31:43Z,2026-02-08,Mid Cap,null,null,0,null,HOLD / NEUTRAL
ENS,6.0235,128,26155480,-0.42,0.06,-12.11,2026-02-08T14:30:00Z,2026-02-08T14:31:43Z,2026-02-08,Small Cap,ETHEREUM NAME SERVICE,2026-02-07,-1,ENS drops its L2.,WARNING (Fakeout)


## Storing the final data in Snowflake Gold Layer

In [0]:
df_snowflake = df_gold.select(
    col("symbol").alias("SYMBOL"),
    col("name").alias("COIN_NAME"),
    col("price").alias("PRICE"),
    col("cmc_rank").alias("COIN_RANK"),
    col("risk_category").alias("RISK_CATEGORY"),
    col("total_daily_score").alias("TOTAL_DAILY_SCORE"),
    col("trade_signal").alias("TRADE_SIGNAL"),
    col("top_headline").alias("TOP_HEADLINE"),
    col("volume_24h").alias("VOLUME_24H"),
    col("percent_change_1h").alias("PERCENT_CHANGE_1H"),
    col("percent_change_24h").alias("PERCENT_CHANGE_24H"),
    col("percent_change_7d").alias("PERCENT_CHANGE_7D"),
    col("market_timestamp").alias("MARKET_TS"),
    col("market_date").alias("MARKET_DATE"),
    col("news_date").alias("NEWS_DATE"),
    current_timestamp().alias("LOAD_DTM")
)


In [0]:
import os

sfOptions = {
  "sfURL": os.environ['SNOWFLAKE_ACCOUNT'],
  "sfUser": os.environ["SNOWFLAKE_USER"],
  "sfPassword": os.environ["SNOWFLAKE_PASSWORD"],
  "sfDatabase": "CRYPTO_ANALYSIS_DB",
  "sfSchema": "GOLD",
  "sfWarehouse": "COMPUTE_WH"
}

In [0]:
df_snowflake.write \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "CRYPTO_ANALYSIS_HIST") \
  .mode("append") \
  .save()
